In [1]:
import fleet as flt


In [ ]:
fleet = flt.AsyncFleet()

environments = await fleet.list_envs()
print("Environments:", len(environments))

# Create a new instance
instance = await fleet.make(flt.InstanceRequest(env_key="hubspot", version="v1.2.7"))
print("New Instance:", instance.instance_id)

environment = await fleet.environment(instance.env_key)
print("Environment Default Version:", environment.default_version)

Environments: 3
[{'name': 'seed', 'type': 'sqlite', 'mode': 'ro', 'label': 'Seed database'}, {'name': 'current', 'type': 'sqlite', 'mode': 'rw', 'label': 'Current database'}, {'name': 'cdp', 'type': 'cdp', 'mode': 'rw', 'label': 'CDP'}]
New Instance: afa358cc
Environment Default Version: v1.2.6


In [5]:
response = await instance.env.reset(flt.ResetRequest(seed=42))
print("Reset response:", response)

print(await instance.env.resources())

sqlite = instance.env.db("current")
print("SQLite:", await sqlite.describe())

Reset response: success=True message='Database reset successfully'
[<fleet.resources.sqlite.AsyncSQLiteResource object at 0x113932570>, <fleet.resources.browser.AsyncBrowserResource object at 0x1156e5bb0>]
SQLite: success=True resource_name='current' tables=[TableSchema(name='activities', sql="CREATE TABLE activities (\n            id INTEGER PRIMARY KEY AUTOINCREMENT,\n            user_id INTEGER NOT NULL, -- User who performed the activity\n            owner_id INTEGER, -- User who owns the activity\n            activity_type TEXT NOT NULL, -- 'email', 'call', 'meeting', 'note', 'stage_change', 'task'\n            subject TEXT,\n            body TEXT,\n            metadata TEXT, -- JSON for additional data like duration, outcome, etc.\n            activity_date DATETIME NOT NULL,\n            createdAt DATETIME DEFAULT CURRENT_TIMESTAMP,\n            FOREIGN KEY (user_id) REFERENCES users(id),\n            FOREIGN KEY (owner_id) REFERENCES users(id)\n        )", columns=[{'cid': 0, '

In [6]:
print("Query:", await sqlite.query("SELECT * FROM users"))

Query: success=True columns=['id', 'email', 'display_name', 'me'] rows=[[1, 'sarah.kim@company.com', 'Sarah Kim', 1], [2, 'john.smith@company.com', 'John Smith', 0], [3, 'emily.chen@company.com', 'Emily Chen', 0], [4, 'michael.brown@company.com', 'Michael Brown', 0], [5, 'lisa.wang@company.com', 'Lisa Wang', 0], [6, 'david.johnson@company.com', 'David Johnson', 0], [7, 'jennifer.davis@company.com', 'Jennifer Davis', 0], [8, 'robert.wilson@company.com', 'Robert Wilson', 0], [9, 'maria.garcia@company.com', 'Maria Garcia', 0], [10, 'james.anderson@company.com', 'James Anderson', 0], [11, 'support.team@company.com', 'Support Team', 0], [12, 'mike.johnson@company.com', 'Mike Johnson', 0], [13, 'edward.jackson@nimbusai.com', 'Edward Jackson', 0], [14, 'sarah.kim@nimbus.ai', 'Sarah Kim', 1], [15, 'betty.alexander@nimbusai.com', 'Betty Alexander', 0], [16, 'raj_patel@nimbusai.ai', 'Raj Patel', 0], [17, 'melissa.carter@nimbusai.com', 'Melissa Carter', 0], [18, 'marcus_johnson@nimbusai.ai', 'Mar

In [ ]:
users_query = await sqlite.query("""
    SELECT id, display_name, email 
    FROM users 
    WHERE display_name IN ('Sarah Kim', 'Raj Patel')
""")
print("Users found:")
for row in users_query.rows:
    print(f"  ID: {row[0]}, Name: {row[1]}, Email: {row[2]}")

# Extract user IDs
sarah_kim_id = next(row[0] for row in users_query.rows if row[1] == 'Sarah Kim')
raj_patel_id = next(row[0] for row in users_query.rows if row[1] == 'Raj Patel')

print(f"\nSarah Kim ID: {sarah_kim_id}")
print(f"Raj Patel ID: {raj_patel_id}")

# Get all current tickets with their properties
tickets_before = await sqlite.query("""
    SELECT id, name, owner_id, properties 
    FROM entries 
    WHERE type = 'ticket'
    ORDER BY id
""")
print(f"\nTotal tickets found: {len(tickets_before.rows)}")

# Show first 5 tickets as example
print("\nFirst 5 tickets (before):")
for i, row in enumerate(tickets_before.rows[:5]):
    print(f"  Ticket {row[0]}: '{row[1]}' - Owner ID: {row[2]}")

data_related_keywords = ['data', 'pipeline', 'integration', 'api', 'sync', 'import', 'export']

tickets_for_raj = []
tickets_for_sarah = []

for ticket in tickets_before.rows:
    ticket_id, ticket_name, current_owner, properties = ticket
    
    # Check if ticket name contains any data-related keywords
    is_data_related = any(keyword in ticket_name.lower() for keyword in data_related_keywords)
    
    if is_data_related:
        tickets_for_raj.append((ticket_id, ticket_name, current_owner))
    else:
        tickets_for_sarah.append((ticket_id, ticket_name, current_owner))

print(f"\nTicket assignment plan:")
print(f"  Tickets for Raj (data-related): {len(tickets_for_raj)}")
print(f"  Tickets for Sarah (others): {len(tickets_for_sarah)}")

print("\nExample SQL updates (first 3 of each):")
print("\nFor Raj (data-related tickets):")
for ticket_id, ticket_name, _ in tickets_for_raj[:3]:
    print(f"  UPDATE entries SET owner_id = {raj_patel_id} WHERE id = {ticket_id} AND type = 'ticket';")
    print(f"    -- Ticket: '{ticket_name}'")

for ticket_id, ticket_name, _ in tickets_for_sarah[:3]:
    print(f"  UPDATE entries SET owner_id = {sarah_kim_id} WHERE id = {ticket_id} AND type = 'ticket';")
    print(f"    -- Ticket: '{ticket_name}'")

# Create validation queries to verify the changes would be correct
print("\n\nValidation queries after updates:")
print(f"""
-- Check Raj's tickets (should all be data-related)
SELECT COUNT(*) as raj_ticket_count 
FROM entries 
WHERE type = 'ticket' AND owner_id = {raj_patel_id};

-- Check Sarah's tickets (should be all others)  
SELECT COUNT(*) as sarah_ticket_count
FROM entries 
WHERE type = 'ticket' AND owner_id = {sarah_kim_id};

-- Verify total tickets unchanged
SELECT COUNT(*) as total_tickets
FROM entries 
WHERE type = 'ticket';
""")

# Summary of changes
changes_summary = {
    'raj_assignments': len([t for t in tickets_for_raj if t[2] != raj_patel_id]),
    'sarah_assignments': len([t for t in tickets_for_sarah if t[2] != sarah_kim_id]),
    'total_tickets': len(tickets_before.rows)
}

print(f"\nSummary of changes needed:")
print(f"  Tickets to reassign to Raj: {changes_summary['raj_assignments']}")
print(f"  Tickets to reassign to Sarah: {changes_summary['sarah_assignments']}")
print(f"  Total tickets in system: {changes_summary['total_tickets']}")


Users found:
  ID: 1, Name: Sarah Kim, Email: sarah.kim@company.com
  ID: 14, Name: Sarah Kim, Email: sarah.kim@nimbus.ai
  ID: 16, Name: Raj Patel, Email: raj_patel@nimbusai.ai

Sarah Kim ID: 1
Raj Patel ID: 16

Total tickets found: 1876

First 5 tickets (before):
  Ticket 278: 'Questions about security compliance' - Owner ID: 60
  Ticket 279: 'Demo data not loading' - Owner ID: 59
  Ticket 280: 'Questions about security compliance' - Owner ID: 33
  Ticket 281: 'Demo data not loading' - Owner ID: 60
  Ticket 282: 'Demo data not loading' - Owner ID: 60

Ticket assignment plan:
  Tickets for Raj (data-related): 590
  Tickets for Sarah (others): 1286

Example SQL updates (first 3 of each):

For Raj (data-related tickets):
  UPDATE entries SET owner_id = 16 WHERE id = 279 AND type = 'ticket';
    -- Ticket: 'Demo data not loading'
  UPDATE entries SET owner_id = 16 WHERE id = 281 AND type = 'ticket';
    -- Ticket: 'Demo data not loading'
  UPDATE entries SET owner_id = 16 WHERE id = 282 

In [ ]:
async def validate_ticket_reassignment(before_snapshot, after_snapshot):
    """
    Validate that tickets are reassigned correctly:
    - Data-related tickets (containing keywords like 'data', 'pipeline', 'integration', etc.) go to Raj
    - All other tickets go to Sarah Kim
    """
    
    # Get user IDs from after snapshot
    sarah_query = await sqlite.query("SELECT id FROM users WHERE display_name = 'Sarah Kim'")
    raj_query = await sqlite.query("SELECT id FROM users WHERE display_name = 'Raj Patel'")
    
    sarah_kim_id = sarah_query.rows[0][0]
    raj_patel_id = raj_query.rows[0][0]
    
    # Define data-related keywords
    data_keywords = ['data', 'pipeline', 'integration', 'api', 'sync', 'import', 'export']
    
    # Get all tickets after changes
    tickets_after = await sqlite.query("""
        SELECT id, name, owner_id 
        FROM entries 
        WHERE type = 'ticket'
    """)
    
    errors = []
    
    # Validate each ticket assignment
    for ticket_id, ticket_name, owner_id in tickets_after.rows:
        is_data_related = any(keyword in ticket_name.lower() for keyword in data_keywords)
        
        if is_data_related:
            # Should be assigned to Raj
            if owner_id != raj_patel_id:
                errors.append(f"Ticket {ticket_id} '{ticket_name}' is data-related but assigned to user {owner_id} instead of Raj ({raj_patel_id})")
        else:
            # Should be assigned to Sarah
            if owner_id != sarah_kim_id:
                errors.append(f"Ticket {ticket_id} '{ticket_name}' is not data-related but assigned to user {owner_id} instead of Sarah ({sarah_kim_id})")
    
    # Check expected changes
    tickets_before = await sqlite.query("""
        SELECT id, name, owner_id 
        FROM entries 
        WHERE type = 'ticket'
    """)
    
    # Count expected changes
    expected_changes = []
    
    for ticket_id, ticket_name, old_owner_id in tickets_before.rows:
        is_data_related = any(keyword in ticket_name.lower() for keyword in data_keywords)
        expected_owner = raj_patel_id if is_data_related else sarah_kim_id
        
        if old_owner_id != expected_owner:
            expected_changes.append({
                'ticket_id': ticket_id,
                'ticket_name': ticket_name,
                'old_owner': old_owner_id,
                'new_owner': expected_owner,
                'reason': 'data-related' if is_data_related else 'general'
            })
    
    # Display validation results
    print(f"Validation Results:")
    print(f"  Total tickets: {len(tickets_after.rows)}")
    print(f"  Expected changes: {len(expected_changes)}")
    print(f"  Validation errors: {len(errors)}")
    
    if errors:
        print("\nErrors found:")
        for error in errors[:5]:  # Show first 5 errors
            print(f"  - {error}")
        if len(errors) > 5:
            print(f"  ... and {len(errors) - 5} more errors")
    else:
        print("\n✓ All tickets assigned correctly!")
    
    # Show change summary
    print("\nChange Summary:")
    raj_tickets = sum(1 for _, _, owner in tickets_after.rows if owner == raj_patel_id)
    sarah_tickets = sum(1 for _, _, owner in tickets_after.rows if owner == sarah_kim_id)
    print(f"  Raj's tickets (data-related): {raj_tickets}")
    print(f"  Sarah's tickets (others): {sarah_tickets}")
    
    return len(errors) == 0


This validation function would verify:
1. All tickets with data-related keywords are assigned to Raj
2. All other tickets are assigned to Sarah Kim
3. No tickets are left unassigned or assigned to wrong person

The function returns True if validation passes, False otherwise.


In [ ]:
task_definition = {
    "id": "reassign-tickets-by-type",
    "problem": "In HubSpot, can you reassign all data-related tickets (containing keywords like 'data', 'pipeline', 'integration', 'api', 'sync', 'import', or 'export') to Raj Patel, and assign all other tickets to me (Sarah Kim)?",
    "category": "task_assignment_and_ownership",
    "difficulty": "medium",
    "solution_approach": """
    1. Find user IDs for Sarah Kim and Raj Patel
    2. Query all tickets (entries with type='ticket')
    3. Categorize tickets based on keywords in their names
    4. Update owner_id for each ticket based on categorization
    5. Validate all tickets are assigned correctly
    """
}

print("Task Definition:")
print(f"ID: {task_definition['id']}")
print(f"Problem: {task_definition['problem']}")
print(f"Category: {task_definition['category']}")
print(f"Difficulty: {task_definition['difficulty']}")
print(f"\nSolution Approach:\n{task_definition['solution_approach']}")

# Show the SQL queries that would implement this task
print("\n\nImplementation SQL:")
print("""
-- Step 1: Find user IDs
SELECT id, display_name FROM users WHERE display_name IN ('Sarah Kim', 'Raj Patel');

-- Step 2: Update data-related tickets to Raj (example with ID 16)
UPDATE entries 
SET owner_id = 16 
WHERE type = 'ticket' 
  AND (LOWER(name) LIKE '%data%' 
    OR LOWER(name) LIKE '%pipeline%'
    OR LOWER(name) LIKE '%integration%'
    OR LOWER(name) LIKE '%api%'
    OR LOWER(name) LIKE '%sync%'
    OR LOWER(name) LIKE '%import%'
    OR LOWER(name) LIKE '%export%');

-- Step 3: Update all other tickets to Sarah (example with ID 14)
UPDATE entries 
SET owner_id = 14 
WHERE type = 'ticket' 
  AND NOT (LOWER(name) LIKE '%data%' 
    OR LOWER(name) LIKE '%pipeline%'
    OR LOWER(name) LIKE '%integration%'
    OR LOWER(name) LIKE '%api%'
    OR LOWER(name) LIKE '%sync%'
    OR LOWER(name) LIKE '%import%'
    OR LOWER(name) LIKE '%export%');
    
-- Step 4: Verify the results
SELECT 
    u.display_name as owner_name,
    COUNT(e.id) as ticket_count,
    GROUP_CONCAT(e.name, ', ') as sample_tickets
FROM entries e
JOIN users u ON e.owner_id = u.id
WHERE e.type = 'ticket'
GROUP BY u.id, u.display_name;
""")


Task Definition:
ID: reassign-tickets-by-type
Problem: In HubSpot, can you reassign all data-related tickets (containing keywords like 'data', 'pipeline', 'integration', 'api', 'sync', 'import', or 'export') to Raj Patel, and assign all other tickets to me (Sarah Kim)?
Category: task_assignment_and_ownership
Difficulty: medium

Solution Approach:

    1. Find user IDs for Sarah Kim and Raj Patel
    2. Query all tickets (entries with type='ticket')
    3. Categorize tickets based on keywords in their names
    4. Update owner_id for each ticket based on categorization
    5. Validate all tickets are assigned correctly
    


Implementation SQL:

-- Step 1: Find user IDs
SELECT id, display_name FROM users WHERE display_name IN ('Sarah Kim', 'Raj Patel');

-- Step 2: Update data-related tickets to Raj (example with ID 16)
UPDATE entries 
SET owner_id = 16 
WHERE type = 'ticket' 
  AND (LOWER(name) LIKE '%data%' 
    OR LOWER(name) LIKE '%pipeline%'
    OR LOWER(name) LIKE '%integration%'